## Deliverable 2. Create a Customer Travel Destinations Map.

In [110]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
from pprint import pprint

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [111]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Smithers,CA,2021-12-21 19:0745,54.7804,-127.1743,7.12,85,90,0.00,light snow
1,1,Kapaa,US,2021-12-21 19:0746,22.0752,-159.3190,73.72,77,90,4.00,light rain
2,2,Warri,NG,2021-12-21 19:0746,5.5167,5.7500,79.68,51,3,6.40,clear sky
3,3,Erenhot,CN,2021-12-21 19:0746,43.6475,111.9767,18.16,62,0,7.56,clear sky
4,4,Saint-Paul,RE,2021-12-21 19:0747,-21.0096,55.2707,78.91,96,100,2.13,light rain


In [112]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?-25
What is the maximum temperature you would like for your trip?40


In [113]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
#Filter the data set to find the cities that fit the criteria
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Smithers,CA,2021-12-21 19:0745,54.7804,-127.1743,7.12,85,90,0.00,light snow
3,3,Erenhot,CN,2021-12-21 19:0746,43.6475,111.9767,18.16,62,0,7.56,clear sky
10,10,Aklavik,CA,2021-12-21 19:0748,68.2191,-135.0107,12.20,85,90,13.80,snow
12,12,Krasnyy Chikoy,RU,2021-12-21 19:0749,50.3643,108.7512,-4.29,100,86,1.16,overcast clouds
13,13,Chifeng,CN,2021-12-21 19:0749,42.2683,118.9636,17.31,58,85,2.98,overcast clouds
15,15,Chokurdakh,RU,2021-12-21 19:0750,70.6333,147.9167,-8.77,97,100,9.93,overcast clouds
17,17,Qaanaaq,GL,2021-12-21 19:0750,77.4840,-69.3632,31.53,70,20,10.13,few clouds
24,24,Xining,CN,2021-12-21 19:0752,36.6167,101.7667,23.92,36,22,1.86,few clouds
33,33,Tasiilaq,GL,2021-12-21 19:0754,65.6145,-37.6368,30.36,61,97,8.84,overcast clouds
39,39,Chern,RU,2021-12-21 19:0756,53.4523,36.9145,-7.38,100,86,5.12,overcast clouds


In [114]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Weather Description    0
dtype: int64

In [115]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Smithers,CA,2021-12-21 19:0745,54.7804,-127.1743,7.12,85,90,0.00,light snow
3,3,Erenhot,CN,2021-12-21 19:0746,43.6475,111.9767,18.16,62,0,7.56,clear sky
10,10,Aklavik,CA,2021-12-21 19:0748,68.2191,-135.0107,12.20,85,90,13.80,snow
12,12,Krasnyy Chikoy,RU,2021-12-21 19:0749,50.3643,108.7512,-4.29,100,86,1.16,overcast clouds
13,13,Chifeng,CN,2021-12-21 19:0749,42.2683,118.9636,17.31,58,85,2.98,overcast clouds
...,...,...,...,...,...,...,...,...,...,...,...
673,673,Boissevain,CA,2021-12-21 19:2441,49.2306,-100.0559,8.06,88,16,11.27,few clouds
674,674,Ballangen,NO,2021-12-21 19:2216,68.3428,16.8315,21.40,62,0,6.91,clear sky
675,675,Borujerd,IR,2021-12-21 19:2441,33.8973,48.7516,36.12,60,0,5.70,clear sky
679,679,Morgaushi,RU,2021-12-21 19:2442,55.9643,46.7743,-2.04,98,97,6.42,overcast clouds


In [116]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Smithers,CA,7.12,light snow,54.7804,-127.1743,
3,Erenhot,CN,18.16,clear sky,43.6475,111.9767,
10,Aklavik,CA,12.20,snow,68.2191,-135.0107,
12,Krasnyy Chikoy,RU,-4.29,overcast clouds,50.3643,108.7512,
13,Chifeng,CN,17.31,overcast clouds,42.2683,118.9636,
15,Chokurdakh,RU,-8.77,overcast clouds,70.6333,147.9167,
17,Qaanaaq,GL,31.53,few clouds,77.4840,-69.3632,
24,Xining,CN,23.92,few clouds,36.6167,101.7667,
33,Tasiilaq,GL,30.36,overcast clouds,65.6145,-37.6368,
39,Chern,RU,-7.38,overcast clouds,53.4523,36.9145,


In [117]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    hotel_loc = f"{lat},{lng}"
    params['location']= hotel_loc
    
    #Use the search term: "lodging" and our latitude and longitude
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url,  params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
 

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [118]:

# 7. Drop the rows where there is no Hotel Name.
null_hotels = hotel_df.loc[hotel_df["Hotel Name"]==""]
hotels_to_drop = null_hotels.index
clean_hotel_df = hotel_df.drop(index = hotels_to_drop)
clean_hotel_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 175 entries, 0 to 683
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 175 non-null    object 
 1   Country              175 non-null    object 
 2   Max Temp             175 non-null    float64
 3   Weather Description  175 non-null    object 
 4   Lat                  175 non-null    float64
 5   Lng                  175 non-null    float64
 6   Hotel Name           175 non-null    object 
dtypes: float64(3), object(4)
memory usage: 10.9+ KB


In [119]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPY_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")
clean_hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 175 entries, 0 to 683
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 175 non-null    object 
 1   Country              175 non-null    object 
 2   Max Temp             175 non-null    float64
 3   Weather Description  175 non-null    object 
 4   Lat                  175 non-null    float64
 5   Lng                  175 non-null    float64
 6   Hotel Name           175 non-null    object 
dtypes: float64(3), object(4)
memory usage: 10.9+ KB


In [122]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Weather Description</dt><dd>{Weather Description}</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]
locations.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 175 entries, 0 to 683
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Lat     175 non-null    float64
 1   Lng     175 non-null    float64
dtypes: float64(2)
memory usage: 4.1 KB


In [123]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations,info_box_content = hotel_info)
fig = gmaps.figure(center = (30.0, 31.0), zoom_level = 1.5)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))